# Basic implementation

Author: Frankie Inguanez <br />
Date: 13/01/2023<br /><br />

A base implementation of a 9x9 sudoku puzzle, that does a raster scan (left to right, top to bottom) to search for an unknown cell, with an ordered guess. This algorithm is a backtracking (brute-force) algorithm.

In [1]:
def to2DArray(n):
    """
    Convert a string to a 2D 9x9 array.
    Arguments:
        n: an 81 character string to be converted to a 2D array representing a 9x9 sudoku grid.
    """
    return [list(map(int, n[i:i+9])) for i in range(0, 81, 9)]

def toStr(puzzle):
    """
    Converts a puzzle to a string
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
    """
    r = ""

    for row in puzzle:
        r += "".join(map(str, row))

    return r

def getColValues(puzzle, col):
    """
    Get column values
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        col: the column number
    """
    lst = []
    for row in puzzle:
        lst.append(row[col])

    return lst

def getBoxValues(puzzle, box):
    """
    Get box values. Boxes are 3x3 sub-grids enumerates from top left in a raster fashion
    0, 1, 2
    3, 4, 5
    6, 7, 8
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        box: the box number
    """
    
    return [puzzle[x][y] for x in range((box//3)*3,((box//3)*3)+3) for y in range((box%3)*3, ((box%3)*3)+3)]

def checkList(lst):
    """
    Check if a list of digits contain all values from 1 to 9
    Arguments:
        lst: the list to check.
    """
    return set(lst) == set(range(1,10))

def isSolved(puzzle):
    """
    Checks if a 9x9 sudoku puzzle has been solved.
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
    """
    # Check rows
    for row in puzzle:
        if not checkList(row):
            return False

    # Check columns
    for i in range(0,9):
        if not checkList(getColValues(puzzle, i)):
            return False;

    # Check box
    for i in range(0,9):
        if not checkList(getBoxValues(puzzle, i)):
            return False;

    return True

def isValid(puzzle, num, pos):
    """
    Checks if a number can be added to a specific position
    Arguments:
        puzzle: A 2D array with a 9x9 sudoku grid.
        num: the number to be inserted
        pos: the row and column position to be considered.
    """

    # Check the row
    for i in range(len(puzzle[0])):
        if puzzle[pos[0]][i]==num and pos[1] != i:
            return False

    # Check the column
    for i in range(len(puzzle[1])):
        if puzzle[i][pos[1]]==num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if puzzle[i][j] == num and (i,j) != pos:
                return False

    return True

def getFileLineCount(fileName: str):
    """
    Get number of lines in a file.
    Arguments:
        fileName: the name of the file to process.
    """
    import mmap

    lines = 0
    with open(fileName, "r+", encoding="utf-8") as f:
        bf = mmap.mmap(f.fileno(), 0)

        while bf.readline():
            lines += 1

    return lines

def saveError(error, errorsFileName: str):
    """
    Saves an error/exception that is raised.
    Arguments:
        error: the Exception that is raised.
        errorsFileName: the file name where the error will be saved.
    """
    try:
        with open(errorsFileName, "a", encoding="utf-8") as ef:
            ef.write("Encountered error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))
    except Exception as e:
        # Failed to save error to file
        print("Failed to save original error to file due to:\n{}\n{}\n{}\n\n".format(type(e), e.args, e))
        print("Original error:\n{}\n{}\n{}\n\n".format(type(error), error.args, error))

In [2]:
# Finds the next empty cell in a raster fashion.
def findRaster(puzzle):
    for row in range(len(puzzle)):
        for col in range(len(puzzle[0])):
            if puzzle[row][col] == 0:
                return (row, col)
            
    return None

In [3]:
# Solves a sudoku 9x9 puzzle in a raster pattern using sequential brute force guessing.
def solveRaster(puzzle):
    find = findRaster(puzzle)

    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if isValid(puzzle, i, (row, col)):
            puzzle[row][col] = i

            if solveRaster(puzzle):
                return True

            puzzle[row][col] = 0

    return False

In [4]:
puzzle = to2DArray('100008509000000000460000000600000000085020047094600302000003090920000700000107005')

print("Solved puzzle!" if solveRaster(puzzle) else "Could not solve puzzle")
puzzle

Solved puzzle!


[[1, 3, 7, 4, 6, 8, 5, 2, 9],
 [2, 5, 8, 3, 1, 9, 4, 7, 6],
 [4, 6, 9, 5, 7, 2, 1, 8, 3],
 [6, 1, 2, 7, 3, 4, 9, 5, 8],
 [3, 8, 5, 9, 2, 1, 6, 4, 7],
 [7, 9, 4, 6, 8, 5, 3, 1, 2],
 [5, 7, 6, 2, 4, 3, 8, 9, 1],
 [9, 2, 1, 8, 5, 6, 7, 3, 4],
 [8, 4, 3, 1, 9, 7, 2, 6, 5]]

In [5]:
def solvePuzzles(puzzlesFileName, statsFileName, errorsFileName, limit):
    import tqdm

    if not limit:
        limit = getFileLineCount(puzzlesFileName)

    i = 0
    hasError = False
    try:
        print("Starting sudoku base solver.")
        
        with open(statsFileName, "w", encoding="utf-8") as sf:
            with open(puzzlesFileName, "r", encoding="utf-8") as pf:
                for line in tqdm.tqdm(pf, total=limit):
                    # Parse the content
                    p = line.strip()

                    puzzle = to2DArray(p)

                    if not solveRaster(puzzle):
                        sf.write("Could not solve puzzle {}".format(line))

                    i+=1

                    if (i>limit):
                        break

    except Exception as e:
        hasError = True
        saveError(e, errorsFileName)

    print("Operation encountered some errors. Check {} for details or script output above.".format(errorsFileName) \
        if hasError else "Sudoku puzzles solved completed successfully.")
    print("Sudoku solver statistics saved in {}".format(statsFileName))


In [6]:
solvePuzzles("puzzles.txt","solverStats.txt", "solverErrors.txt", 2)

Starting sudoku base solver.


100%|██████████| 2/2 [00:00<00:00, 285.64it/s]

Sudoku puzzles solved completed successfully.
Sudoku solver statistics saved in solverStats.txt
